# Cloud chornicles - Microsoft Azure AutoML model 

In [1]:
#importing required libraries
import pandas as pd
from azureml.core import Dataset
from datetime import datetime
from dateutil.relativedelta import relativedelta
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

In [55]:
#installing require packages
!pip install --upgrade azureml-sdk azureml-widgets

Requirement already up-to-date: azureml-sdk in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.24.0)
Requirement already up-to-date: azureml-widgets in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.24.0)


In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()

print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.24.0 to work with cloudchronicles


In [3]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()
default_ds

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-14994ef0-dd19-46ed-9811-bb6743361672",
  "account_name": "cloudchronicle3731318223",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [4]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

Datasets:
	 sbadata version 1


Remove some of the columns that you won't need for training or additional feature building.

In [5]:
#import dataset
#use version 3
sba_df = Dataset.get_by_name(ws, 'sbadata', version = 1)
sba_df = sba_df.to_pandas_dataframe()

In [6]:
sba_df.columns

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursemnetFY',
       'DisbursementGross', 'BalanceGross', 'MIS_Status', 'ChgOffPrinGr',
       'GrAppv', 'SBA_Appv', 'Percentage_sabapp', 'default_status', 'Industry',
       'IsFranchise', 'RealEstate', 'Recession', 'same_bankstate',
       'Daystodisbursement'],
      dtype='object')

In [7]:
sba_df.isnull().sum()

LoanNr_ChkDgt              0
Name                       0
City                       0
State                      0
Zip                        0
Bank                       0
BankState                  0
NAICS                      0
ApprovalDate               0
ApprovalFY                 0
Term                       0
NoEmp                      0
NewExist                   0
CreateJob                  0
RetainedJob                0
FranchiseCode              0
UrbanRural                 0
RevLineCr                  0
LowDoc                     0
ChgOffDate            478590
DisbursementDate           0
DisbursemnetFY             0
DisbursementGross          0
BalanceGross               0
MIS_Status                 0
ChgOffPrinGr               0
GrAppv                     0
SBA_Appv                   0
Percentage_sabapp          0
default_status             0
Industry                   0
IsFranchise                0
RealEstate                 0
Recession                  0
same_bankstate

In [8]:
sba_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591176 entries, 0 to 591175
Data columns (total 36 columns):
LoanNr_ChkDgt         591176 non-null int64
Name                  591176 non-null object
City                  591176 non-null object
State                 591176 non-null object
Zip                   591176 non-null int64
Bank                  591176 non-null object
BankState             591176 non-null object
NAICS                 591176 non-null int64
ApprovalDate          591176 non-null datetime64[ns]
ApprovalFY            591176 non-null int64
Term                  591176 non-null int64
NoEmp                 591176 non-null int64
NewExist              591176 non-null int64
CreateJob             591176 non-null int64
RetainedJob           591176 non-null int64
FranchiseCode         591176 non-null int64
UrbanRural            591176 non-null int64
RevLineCr             591176 non-null int64
LowDoc                591176 non-null int64
ChgOffDate            112586 non-null d

In [9]:
sba_df.drop(columns = ['LoanNr_ChkDgt','City','Name','State','Zip','Bank','BankState','NAICS','ApprovalDate','ApprovalFY','FranchiseCode','ChgOffDate','DisbursementDate','DisbursemnetFY','MIS_Status','SBA_Appv','ChgOffPrinGr'], inplace=True)

In [10]:
sba_df.columns

Index(['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'UrbanRural',
       'RevLineCr', 'LowDoc', 'DisbursementGross', 'BalanceGross', 'GrAppv',
       'Percentage_sabapp', 'default_status', 'Industry', 'IsFranchise',
       'RealEstate', 'Recession', 'same_bankstate', 'Daystodisbursement'],
      dtype='object')

In [11]:
#Setting up workspace
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

In [12]:
#creating training and testing dataset
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(sba_df, test_size=0.2, random_state=223)

In [13]:
x_train

,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,Percentage_sabapp,default_status,Industry,IsFranchise,RealEstate,Recession,same_bankstate,Daystodisbursement
300456,132,1,2,0,0,0,0,0,21000,0,21000,83.000000,0,Wholesale_trade,0,0,0,1,89
272534,180,2,1,0,0,2,0,0,650000,0,650000,68.680000,0,Other_no_pub,0,0,1,0,15
358608,240,14,1,0,0,0,0,0,670000,0,670000,80.100000,0,RE/Rental/Lease,0,1,1,1,100
191730,180,1,2,0,0,2,0,0,160000,0,160000,75.000000,0,Wholesale_trade,0,0,1,1,35
190533,29,1,2,0,1,1,1,0,23283,0,10000,21.474896,1,Educational,0,0,1,0,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527143,84,6,1,0,0,0,0,1,60000,0,60000,90.000000,0,Retail_trade,0,0,0,1,34
566430,60,15,1,0,0,0,0,1,89000,0,89000,18.000000,0,Retail_trade,0,0,0,0,85
540847,180,2,1,0,0,0,0,0,250200,0,250200,75.000000,0,Ag/For/Fish/Hunt,0,0,1,1,347
158398,84,1,2,0,1,1,0,0,25000,0,25000,50.000000,0,Prof/Science/Tech,0,0,1,0,23


In [14]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

In [15]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             debug_log='automated_ml_errors.log',
                             training_data=x_train,
                             label_column_name= 'default_status',
                             **automl_settings)

In [16]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, 'bank')


In [17]:
local_run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_2a1388d9-c9a6-452f-9ef3-cdaca48df016

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

**************************************************

Run.fail() is deprecated, use Run.tag() to mark run as failed or use Run.cancel() to mark the run as canceled
Run.fail() will be removed shortly


                                               0:00:12          nan    0.9746
ERROR: FitException:
	Message: /anaconda/envs/azureml_py36/lib/libxgboost.so: undefined symbol: XGDMatrixSetDenseInfo
	InnerException: AttributeError: /anaconda/envs/azureml_py36/lib/libxgboost.so: undefined symbol: XGDMatrixSetDenseInfo
	ErrorResponse 
{
    "error": {
        "code": "SystemError",
        "message": "Encountered an internal AutoML error. Error Message/Code: FitException. Additional Info: FitException:\n\tMessage: /anaconda/envs/azureml_py36/lib/libxgboost.so: undefined symbol: XGDMatrixSetDenseInfo\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"message\": \"/anaconda/envs/azureml_py36/lib/libxgboost.so: undefined symbol: XGDMatrixSetDenseInfo\",\n        \"target\": \"Xgboost\",\n        \"reference_code\": \"Xgboost\"\n    }\n}",
        "details_uri": "https://docs.microsoft.com/azure/machine-learning/resource-known-issues#automated-machine-learning",
        "

In [18]:
from azureml.widgets import RunDetails
RunDetails(local_run).show() 

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Retrieve the best model

Select the best model from your iterations. The `get_output` function returns the best run and the fitted model for the last fit invocation. By using the overloads on `get_output`, you can retrieve the best run and fitted model for any logged metric or a particular iteration.

In [19]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: bank,
Id: AutoML_2a1388d9-c9a6-452f-9ef3-cdaca48df016_0,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MaxAbsScaler', MaxAbsScaler(copy...
                 LightGBMClassifier(boosting_type='gbdt', class_weight=None,
                                    colsample_bytree=1.0,
                                    importance_type='split', learning_rate=0.1,
                                    max_depth=-1, min_child_samples=20,


### Test the best model accuracy

In [20]:
y_test = x_test.pop("default_status")

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

[1 0 0 0 0 1 0 0 0 0]


In [21]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

0.23838419393547292

In [22]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.3069438099588853

Model Accuracy:
0.6930561900411147
